In [ ]:
import os
import json

import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
from torchvision.utils import draw_bounding_boxes
from torchvision.io import read_image

In [ ]:
os.makedirs("visualization/frames/", exist_ok=True)
os.makedirs("visualization/annotated_frames/", exist_ok=True)

In [ ]:
def video2frame(
    tfile,frame_width, frame_height, interval):
    """
    Extract frame from video by interval
    :param video_src_path: video src path
    :param video:　video file name
    :param frame_width:　frame width
    :param frame_height:　frame height
    :param interval:　interval for frame to extract
    :return:　list of numpy.ndarray 
    """
    video_frames = []
    cap = cv2.VideoCapture(tfile)
    frame_index = 0
    frame_count = 0
    if cap.isOpened():
        success = True
    else:
        success = False
        print("Read failed!")

    while success:
        success, frame = cap.read()

        if frame_index % interval == 0:
            print("---> Reading the %d frame:" % frame_index, success)
            resize_frame = cv2.resize(
                frame, (frame_width, frame_height), interpolation=cv2.INTER_AREA
            )
            video_frames.append(resize_frame)
            frame_count += 1

        frame_index += 1

    cap.release()
    
    print('Number of frames')
    print(frame_count)
    return video_frames

In [ ]:
video_frames = video2frame('videos/ducks.mp4', 1024, 1024, 30)
for i in range(len(video_frames)):
    cv2.imwrite(f"visualization/frames/image-{i}.jpg", video_frames[i])

In [ ]:
plt.rcParams["savefig.bbox"] = 'tight'


def save(imgs, img_num):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
    plt.savefig(f"visualization/annotated_frames/{str(img_num).zfill(3)}.jpg")

In [ ]:
with open("visualization/output.json", "r") as read_file:
    data = json.load(read_file)


In [ ]:
for i in range(len(data)):
    score_threshold = .9
    scores = torch.from_numpy(np.array(data[i]['scores']))
    boxes = torch.from_numpy(np.array(data[i]['boxes']))

    birds_with_boxes = [
        draw_bounding_boxes(read_image(f'visualization/frames/image-{i}.jpg'), boxes=boxes[scores > score_threshold], width=10)
     ]
    save(birds_with_boxes, i)

In [ ]:
import glob
from PIL import Image

# filepaths
fp_in = "visualization/annotated_frames/*.jpg"
fp_out = "visualization/annotated_frames/birds.gif"


img, *imgs = [Image.open(f) for f in sorted(glob.glob(fp_in))]
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=200, loop=0)
